# ***CUARTO MODELO Y FINAL***

In [23]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import linregress
import random
import warnings

# Ignorar advertencias de yfinance y otras librerías para una salida más limpia
warnings.filterwarnings("ignore")

# --- Configuración Global y Parámetros por Defecto ---
config = {
    "symbol": "BTC-USD",
    "initial_portfolio": 10000,
    "strategy_name": "ichimoku",
    "risk_per_trade_pct": 1.0,
    "position_sizing_mode": "kelly_fractional", # 'fixed', 'kelly_fractional'
    "kelly_fraction": 0.5,
    "take_profit_multiple": 2.0,
    "atr_period": 14,
    "trailing_stop_percent": 1.5,
    "use_ml_filter": True,
    "commission_pct": 0.1, # Comisión del 0.1% por operación
    "tax_pct": 10.0, # Impuesto del 10% sobre las ganancias
    "time_period": "1y",
    "time_interval": "1d",
    "advanced_analysis": "none", # 'genetic_algorithm', 'sensitivity', 'walk_forward'
    "walk_forward_training_period": 120,
    "walk_forward_testing_period": 30,
    # Parámetros de estrategias
    "ema_short_period": 12, "ema_long_period": 26,
    "ichimoku_tenkan": 9, "ichimoku_kijun": 26, "ichimoku_senkou": 52,
    "bollinger_period": 20, "bollinger_dev": 2.0,
    # Parámetros de ML
    "ml_features": ['ema_short', 'ema_long', 'rsi', 'macd', 'macd_signal', 'bb_high', 'bb_low', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b']
}

# --- Menú de Configuración Interactivo ---
def setup_menu():
    """
    Muestra un menú interactivo y claro para configurar el bot de trading.
    """
    print("--- Configuración del Backtesting Definitivo ---")
    print("Por favor, ingrese los siguientes parámetros. Presione ENTER para usar los valores por defecto.")
    print("-" * 50)

    try:
        symbol = input(f"Símbolo del Activo (ej: BTC-USD, AAPL)\n[default: {config['symbol']}]: ").upper() or config['symbol']
        config['symbol'] = symbol

        initial_portfolio_input = input(f"Capital Inicial [default: {config['initial_portfolio']}]: ")
        if initial_portfolio_input:
            config['initial_portfolio'] = float(initial_portfolio_input)

        strategy_choice = input("Seleccione una estrategia:\n1. Crossover de EMA (para mercados en tendencia)\n2. Bollinger Bands (para reversión a la media)\n3. Ichimoku Cloud (análisis completo de tendencia)\n(1/2/3) [default: 3]: ")
        if strategy_choice == "1":
            config['strategy_name'] = "crossover"
        elif strategy_choice == "2":
            config['strategy_name'] = "bollinger"
        elif strategy_choice == "3":
            config['strategy_name'] = "ichimoku"

        sizing_choice = input("Seleccione el método de gestión de capital:\n1. Riesgo Fijo por Operación (simple)\n2. Criterio de Kelly Fraccional (avanzado, basado en el historial)\n(1/2) [default: 2]: ")
        if sizing_choice == "1":
            config['position_sizing_mode'] = "fixed"
            risk_input = input(f"Riesgo por operación (%) [default: {config['risk_per_trade_pct']}]: ")
            if risk_input:
                config['risk_per_trade_pct'] = float(risk_input)
        else:
            kelly_input = input(f"Fracción de Kelly (0.1 a 1.0) [default: {config['kelly_fraction']}]: ")
            if kelly_input:
                config['kelly_fraction'] = float(kelly_input)


        # Opciones avanzadas excluyentes
        print("\n--- Análisis Avanzado (Seleccione solo uno) ---")
        advanced_choice = input("Seleccione una opción de análisis avanzado:\n1. Algoritmo Genético (optimiza parámetros automáticamente)\n2. Análisis de Sensibilidad (mapa de calor)\n3. Análisis Walk-Forward (validación rigurosa)\n4. Sin Análisis Avanzado\n(1/2/3/4) [default: 4]: ")
        if advanced_choice == "1":
            config['advanced_analysis'] = "genetic_algorithm"
        elif advanced_choice == "2":
            config['advanced_analysis'] = "sensitivity"
        elif advanced_choice == "3":
            config['advanced_analysis'] = "walk_forward"
        else:
            config['advanced_analysis'] = "none"

        use_ml = input("¿Activar Filtro de Machine Learning para señales? (s/n) [default: s]: ").lower()
        config['use_ml_filter'] = (use_ml != 'n')

    except ValueError:
        print("Entrada no válida. Se usarán los valores por defecto.")

    return config['symbol']

# --- Funciones de Gestión de Riesgo y Capital ---
def calculate_position_size(portfolio_value, entry_price, stop_loss_price, win_rate=None, avg_win_loss=None):
    price_risk = abs(entry_price - stop_loss_price)
    if price_risk <= 1e-6: # Usar una pequeña tolerancia para evitar división por cero
        return 0.0

    if config['position_sizing_mode'] == 'fixed':
        risk_amount = portfolio_value * (config['risk_per_trade_pct'] / 100)
        units = risk_amount / price_risk

    elif config['position_sizing_mode'] == 'kelly_fractional' and win_rate is not None and avg_win_loss is not None:
        if avg_win_loss <= 0 or win_rate <= 0: # Corregido para Kelly > 0
            # Fallback a fijo si Kelly no es calculable o no rentable
            risk_amount = portfolio_value * (config['risk_per_trade_pct'] / 100)
            units = risk_amount / price_risk
            return units

        kelly_fraction = win_rate - ((1 - win_rate) / avg_win_loss)
        if kelly_fraction <= 0: # Kelly negativo
             # Fallback a fijo si Kelly es negativo
            risk_amount = portfolio_value * (config['risk_per_trade_pct'] / 100)
            units = risk_amount / price_risk
            return units

        kelly_fraction *= config['kelly_fraction']
        risk_amount = portfolio_value * kelly_fraction
        units = risk_amount / price_risk

    else: # Fallback a fijo si Kelly no tiene datos suficientes
        risk_amount = portfolio_value * (config['risk_per_trade_pct'] / 100)
        units = risk_amount / price_risk

    capital_needed = units * entry_price
    if capital_needed > portfolio_value * 0.95: # No arriesgar más del 95% del portafolio en una sola operación
        units = (portfolio_value * 0.95) / entry_price
        print(f"⚠️ Reduciendo tamaño de posición para no exceder el capital disponible.")


    return units

# --- Funciones de Estrategias de Trading (mejoradas) ---
def calculate_indicators(data, params):
    """ Calcula todos los indicadores técnicos necesarios. """
    # Ensure price, high, and low are pandas Series
    price_series = data['price'].squeeze()
    high_series = data['High'].squeeze()
    low_series = data['Low'].squeeze()


    try:
        data['atr'] = ta.volatility.average_true_range(high_series, low_series, price_series, window=params['atr_period'])
    except Exception as e:
        print(f"Error calculating ATR: {e}")
        data['atr'] = np.nan # Assign NaN on error


    if params['strategy_name'] == 'crossover':
        try:
            data['ema_short'] = ta.trend.ema_indicator(price_series, window=params.get('ema_short_period', config['ema_short_period']))
            data['ema_long'] = ta.trend.ema_indicator(price_series, window=params.get('ema_long_period', config['ema_long_period']))
        except Exception as e:
            print(f"Error calculating EMA: {e}")
            data['ema_short'] = np.nan
            data['ema_long'] = np.nan


    elif params['strategy_name'] == 'bollinger':
        try:
            bollinger = ta.volatility.BollingerBands(price_series, window=params.get('bollinger_period', config['bollinger_period']), window_dev=params.get('bollinger_dev', config['bollinger_dev']))
            data['bb_high'] = bollinger.bollinger_hband()
            data['bb_low'] = bollinger.bollinger_lband()
        except Exception as e:
            print(f"Error calculating Bollinger Bands: {e}")
            data['bb_high'] = np.nan
            data['bb_low'] = np.nan


    elif params['strategy_name'] == 'ichimoku':
        try:
            ichimoku = ta.trend.IchimokuIndicator(high=high_series, low=low_series, window1=params.get('ichimoku_tenkan', config['ichimoku_tenkan']), window2=params.get('ichimoku_kijun', config['ichimoku_kijun']), window3=params.get('ichimoku_senkou', config['ichimoku_senkou']))
            data['tenkan_sen'] = ichimoku.ichimoku_conversion_line()
            data['kijun_sen'] = ichimoku.ichimoku_base_line()
            data['senkou_span_a'] = ichimoku.ichimoku_a()
            data['senkou_span_b'] = ichimoku.ichimoku_b()
        except Exception as e:
            print(f"Error calculating Ichimoku: {e}")
            data['tenkan_sen'] = np.nan
            data['kijun_sen'] = np.nan
            data['senkou_span_a'] = np.nan
            data['senkou_span_b'] = np.nan


    # Indicadores comunes para ML
    try:
        data['rsi'] = ta.momentum.rsi(price_series, window=config.get('rsi_period', 14))
    except Exception as e:
        print(f"Error calculating RSI: {e}")
        data['rsi'] = np.nan

    try:
        macd_obj = ta.trend.MACD(price_series, window_fast=config.get('macd_fast_period', 12), window_slow=config.get('macd_slow_period', 26), window_sign=config.get('macd_signal_period', 9))
        data['macd'], data['macd_signal'] = macd_obj.macd(), macd_obj.macd_signal()
    except Exception as e:
        print(f"Error calculating MACD: {e}")
        data['macd'] = np.nan
        data['macd_signal'] = np.nan


    return data

def check_crossover_strategy(data, i, params):
    buy_signal = (data['ema_short'].iloc[i] > data['ema_long'].iloc[i] and data['ema_short'].iloc[i-1] <= data['ema_long'].iloc[i-1])
    sell_signal = (data['ema_short'].iloc[i] < data['ema_long'].iloc[i] and data['ema_short'].iloc[i-1] >= data['ema_long'].iloc[i-1])
    return buy_signal, sell_signal

def check_bollinger_strategy(data, i, params):
    buy_signal = data['price'].iloc[i] < data['bb_low'].iloc[i]
    sell_signal = data['price'].iloc[i] > data['bb_high'].iloc[i]
    return buy_signal, sell_signal

def check_ichimoku_strategy(data, i, params):
    # Cruce de Tenkan sobre Kijun por encima de la nube
    buy_signal = (data['tenkan_sen'].iloc[i] > data['kijun_sen'].iloc[i] and
                  data['tenkan_sen'].iloc[i-1] <= data['kijun_sen'].iloc[i-1] and
                  data['price'].iloc[i] > data['senkou_span_a'].iloc[i] and
                  data['price'].iloc[i] > data['senkou_span_b'].iloc[i])

    # Cruce de Tenkan bajo Kijun por debajo de la nube
    sell_signal = (data['tenkan_sen'].iloc[i] < data['kijun_sen'].iloc[i] and
                   data['tenkan_sen'].iloc[i-1] >= data['kijun_sen'].iloc[i-1] and
                   data['price'].iloc[i] < data['senkou_span_a'].iloc[i] and
                   data['price'].iloc[i] < data['senkou_span_b'].iloc[i])

    return buy_signal, sell_signal

def train_ml_model(data, features_list):
    """ Trains a Logistic Regression model using available features. """
    # Filter features to include only those present in the data
    available_features = [f for f in features_list if f in data.columns]
    if not available_features:
        print("Advertencia: No hay características de ML disponibles en los datos.")
        return None, None

    # Create a combined DataFrame for features and target
    features_and_target = data[available_features].copy()
    # Calculate target *before* dropping NaNs to ensure alignment
    features_and_target['target'] = (data['price'].shift(-1) > data['price']).astype(int)

    # Drop NaNs from the combined DataFrame
    features_and_target.dropna(inplace=True)

    if features_and_target.empty:
        print("Advertencia: El dataframe de características y objetivo de ML está vacío después de eliminar NaNs.")
        return None, None

    features_df = features_and_target[available_features]
    target_series = features_and_target['target']

    # Ensure there's at least one sample for splitting
    if len(features_df) < 2:
         print("Advertencia: Datos insuficientes para entrenar el modelo de ML.")
         return None, None

    # Ensure there's at least one sample for each class in the target
    if target_series.nunique() < 2:
        print("Advertencia: La serie objetivo de ML no tiene suficientes clases para estratificación.")
        test_size_val = max(0.2, 1 / len(features_df)) # Adjust test size
        X_train, X_test, y_train, y_test = train_test_split(features_df, target_series, test_size=test_size_val, random_state=42)
    else:
        X_train, X_test, y_train, y_test = train_test_split(features_df, target_series, test_size=0.2, random_state=42, stratify=target_series)


    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    model = LogisticRegression(random_state=42, solver='liblinear')
    model.fit(X_train_scaled, y_train)
    return model, scaler

# --- Functions for Advanced Analysis ---
def run_optimization_ga(data, params):
    """
    Optimización de parámetros usando un algoritmo genético.
    """
    print("\nIniciando optimización con Algoritmo Genético...")
    # Definir el rango de parámetros para optimizar
    if params['strategy_name'] == 'crossover':
        param_ranges = {'ema_short_period': range(5, 20), 'ema_long_period': range(20, 50)}
    elif params['strategy_name'] == 'bollinger':
        param_ranges = {'bollinger_period': range(10, 30), 'bollinger_dev': [i / 10.0 for i in range(15, 30)]}
    elif params['strategy_name'] == 'ichimoku':
        param_ranges = {'ichimoku_tenkan': range(7, 12), 'ichimoku_kijun': range(20, 30)}
    else:
        print("Algoritmo Genético no disponible para esta estrategia.")
        return params

    population_size = 20
    generations = 5

    population = [
        {p: random.choice(list(param_ranges[p])) for p in param_ranges}
        for _ in range(population_size)
    ]

    best_chromosome = None
    best_performance = -np.inf

    for gen in range(generations):
        fitness_scores = []
        # Calculate indicators once for the data used in this generation's simulations
        data_with_indicators = calculate_indicators(data.copy(), params) # Calculate with base params for common indicators
        for chromosome in population:
            test_params = {**params, **chromosome}
             # Recalculate strategy-specific indicators with test_params
            current_data = calculate_indicators(data_with_indicators.copy(), test_params)
            results = run_simulation(current_data, test_params, is_optimization=True)
            fitness = results.get('sharpe_ratio', -np.inf) # Use get with default for safety
            fitness_scores.append(fitness)
            if fitness > best_performance:
                best_performance = fitness
                best_chromosome = chromosome

        # Selección, Crossover y Mutación
        # Handle cases where fitness_scores might be empty or contain only -inf
        if not fitness_scores or all(f == -np.inf for f in fitness_scores):
             print(f"Generación {gen}: No hay resultados válidos para la selección.")
             # Optionally, repopulate or exit if no progress
             continue

        sorted_population = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
        new_population = sorted_population[:int(population_size * 0.5)] # Élite

        while len(new_population) < population_size:
            parent1 = random.choice(new_population)
            parent2 = random.choice(new_population)

            child = {}
            for p in param_ranges:
                if random.random() > 0.5:
                    child[p] = parent1[p]
                else:
                    child[p] = parent2[p]

            # Mutación
            for p in param_ranges:
                if random.random() < 0.1: # Tasa de mutación
                    child[p] = random.choice(list(param_ranges[p]))
            new_population.append(child)

        population = new_population

    print(f"\n✅ Optimización terminada. Mejor resultado (Ratio de Sharpe: {best_performance:.2f}) con los parámetros:")
    print(best_chromosome)
    return best_chromosome if best_chromosome else params # Return best or original params

def run_sensitivity_analysis(data, params):
    """
    Realiza un análisis de sensibilidad y genera un mapa de calor.
    """
    print("\nIniciando Análisis de Sensibilidad (Mapa de Calor)...")
    if params['strategy_name'] == 'crossover':
        param1_name, param2_name = 'ema_short_period', 'ema_long_period'
        param1_range = range(5, 20)
        param2_range = range(20, 50)
    elif params['strategy_name'] == 'bollinger':
        param1_name, param2_name = 'bollinger_period', 'bollinger_dev'
        param1_range = range(10, 30, 2)
        param2_range = [i / 10.0 for i in range(10, 40, 2)]
    elif params['strategy_name'] == 'ichimoku':
        param1_name, param2_name = 'ichimoku_tenkan', 'ichimoku_kijun'
        param1_range = range(7, 12)
        param2_range = range(20, 30)
    else:
        print("Análisis de sensibilidad no disponible para esta estrategia.")
        return

    # Validar que los rangos no estén vacíos. Esta fue tu solución.
    if not param1_range or not param2_range:
        print("Error: Los rangos de parámetros para el análisis de sensibilidad están vacíos. No se puede generar el mapa de calor.")
        print("param1_range:", list(param1_range))
        print("param2_range:", list(param2_range))
        return

    # Corrección del error: Usar una matriz de numpy para inicializar el DataFrame
    index_list = list(param1_range)
    columns_list = list(param2_range)

    results_matrix = pd.DataFrame(
        np.zeros((len(index_list), len(columns_list))),
        index=index_list,
        columns=columns_list
    )

    results_matrix.index.name = param1_name
    results_matrix.columns.name = param2_name

    # Calculate common indicators once
    data_with_indicators = calculate_indicators(data.copy(), params)

    initial_portfolio_for_sensitivity = float(params.get('initial_portfolio', config['initial_portfolio'])) # Ensure initial portfolio is float

    for i, p1 in enumerate(param1_range):
        for j, p2 in enumerate(param2_range):
            test_params = {**params, param1_name: p1, param2_name: p2}
             # Recalculate strategy-specific indicators with test_params
            current_data = calculate_indicators(data_with_indicators.copy(), test_params)
            results = run_simulation(current_data, test_params, is_optimization=True) # is_optimization=True for sensitivity

            # Ensure total_return is calculated correctly, handling potential None
            sim_total_return = results.get('total_return')
            if sim_total_return is None:
                 # Fallback to calculating return manually if not present
                final_port = results.get('final_portfolio', initial_portfolio_for_sensitivity)
                if initial_portfolio_for_sensitivity is not None and initial_portfolio_for_sensitivity > 0:
                     sim_total_return = ((float(final_port) - initial_portfolio_for_sensitivity) / initial_portfolio_for_sensitivity) * 100 # Ensure final_port is float
                else:
                    sim_total_return = 0.0 # Default to 0.0 if initial portfolio is invalid

            results_matrix.loc[p1, p2] = float(sim_total_return) # Ensure assignment is float


    plt.style.use('dark_background')
    plt.figure(figsize=(10, 8))
    sns.heatmap(results_matrix, cmap='viridis', annot=True, fmt=".1f")
    plt.title(f'Mapa de Calor de Retorno Total para {params["strategy_name"].capitalize()}')
    plt.xlabel(param2_name.replace('_', ' ').title())
    plt.ylabel(param1_name.replace('_', ' ').title())
    plt.tight_layout()
    plt.savefig('sensitivity_heatmap.png')
    print("Mapa de calor de sensibilidad guardado como 'sensitivity_heatmap.png'")

def run_walk_forward_analysis(data, params):
    """
    Realiza un backtest walk-forward.
    """
    print("\nIniciando Análisis Walk-Forward...")

    training_period = params['walk_forward_training_period']
    testing_period = params['walk_forward_testing_period']

    start_index = 0
    total_walk_forward_pnl = []
    # Ensure initial_portfolio is a float and local to the function
    initial_portfolio_wf_local = float(config.get('initial_portfolio', 10000))
    walk_forward_equity = [float(initial_portfolio_wf_local)] # Ensure the first element is also explicitly cast to float
    print(f"Debug: Initial Walk-Forward Equity: {walk_forward_equity[0]} (Type: {type(walk_forward_equity[0])})") # Debug print initial equity


    if data.empty or len(data) < training_period + testing_period:
        print("Datos insuficientes para realizar un análisis Walk-Forward con los períodos especificados.")
        return

    while start_index + training_period + testing_period <= len(data):
        # Slice data and ensure 'Date' column is kept
        train_data = data.iloc[start_index : start_index + training_period].copy()
        test_data = data.iloc[start_index + training_period : start_index + training_period + testing_period].copy()

        # Recalculate indicators on sliced data and reset index to ensure 'Date' column
        train_data = calculate_indicators(train_data, params)
        train_data.dropna(inplace=True)
        train_data.reset_index(drop=False, inplace=True)

        test_data = calculate_indicators(test_data, params)
        test_data.dropna(inplace=True)
        test_data.reset_index(drop=False, inplace=True)


        # Ensure 'Date' column exists before accessing it and data is not empty
        train_start_date = train_data.iloc[0]['Date'].date() if 'Date' in train_data.columns and not train_data.empty and isinstance(train_data.iloc[0]['Date'], pd.Timestamp) else "N/A"
        test_end_date = test_data.iloc[-1]['Date'].date() if 'Date' in test_data.columns and not test_data.empty and isinstance(test_data.iloc[-1]['Date'], pd.Timestamp) else "N/A"
        print(f"\n--- Walk-Forward Window: {train_start_date} to {test_end_date} ---")


        # Optimize on the training window
        best_params = run_optimization_ga(train_data, params)

        # Test on the forward window using the optimized parameters
        # Use the test_data with indicators calculated above
        results = run_simulation(test_data, {**params, **best_params}, is_walk_forward=True)

        # Accumulate PNL and update equity
        test_trades = results.get('trades_log', [])
        # Ensure window_pnl is a float, default to 0.0 if sum is problematic or None
        window_pnl = float(sum([trade['pnl'] for trade in test_trades])) if test_trades and all(isinstance(t.get('pnl'), (int, float)) for t in test_trades) else 0.0

        # Ensure the last element of walk_forward_equity is a float before adding
        last_equity = float(walk_forward_equity[-1]) if walk_forward_equity and walk_forward_equity[-1] is not None else initial_portfolio_wf_local
        walk_forward_equity.append(float(last_equity + window_pnl)) # Explicitly cast the sum to float


        start_index += testing_period

    final_portfolio = walk_forward_equity[-1]
    # Ensure final_portfolio_wf is a float and initial_portfolio_wf_local is a float before calculation
    final_portfolio_wf = float(final_portfolio) if final_portfolio is not None else initial_portfolio_wf_local
    initial_portfolio_calc = float(initial_portfolio_wf_local) if initial_portfolio_wf_local is not None else 0.0

    # Calculate total return, handling division by zero and ensuring float types
    total_return = 0.0
    try:
        # Explicitly cast to float and check for non-zero initial portfolio before division
        initial_for_division = float(initial_portfolio_calc)
        final_for_subtraction = float(final_portfolio_wf)

        print(f"Debug: Calculating total_return. Final portfolio: {final_for_subtraction}, Initial portfolio: {initial_for_division}") # Debug print

        if initial_for_division > 1e-9: # Use a small epsilon to check for near-zero
             total_return = ((final_for_subtraction - initial_for_division) / initial_for_division) * 100
        else:
            # Handle cases where initial portfolio is zero or very close to zero
            if final_for_subtraction > initial_for_division:
                total_return = float('inf') # Represents infinite return from zero capital
            elif final_for_subtraction < initial_for_division:
                total_return = -100.0 # Represents complete loss from zero capital (or near zero)
            else:
                total_return = 0.0 # No change from zero capital


    except (ValueError, TypeError) as e:
        print(f"Error calculating total return: {e}. Final portfolio: {final_for_subtraction}, Initial portfolio: {initial_for_division}")
        total_return = 0.0 # Default to 0.0 on error


    # Calculate metrics for the combined walk-forward results
    equity_curve = pd.Series(walk_forward_equity)
    peak = equity_curve.expanding(min_periods=1).max()
    drawdown = ((equity_curve - peak) / peak)
    max_drawdown = drawdown.min()

    daily_returns = equity_curve.pct_change().dropna()
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252) if daily_returns.std() > 1e-6 else 0.0 # Avoid division by near-zero
    annualized_return = (1 + daily_returns.mean()) ** 252 - 1
    calmar_ratio = annualized_return / abs(max_drawdown) if abs(max_drawdown) > 1e-6 else float('inf') # Avoid division by near-zero


    print("-" * 50)
    print("Análisis Walk-Forward Terminado.")
    print(f"Portafolio inicial: ${initial_portfolio_wf_local:.2f}") # Use initial_portfolio_wf_local for printing
    print(f"Portafolio final:   ${final_portfolio_wf:.2f}")
    print(f"Retorno Total: {total_return:.2f}%")
    print(f"Retorno Anualizado (estimado): {annualized_return:.2f}%")
    print(f"Drawdown Máximo:    {max_drawdown:.2f}%")
    print(f"Ratio de Calmar:    {calmar_ratio:.2f}")
    print(f"Ratio de Sharpe:    {sharpe_ratio:.2f}")

    # Optional: Plot Walk-Forward Equity Curve
    plt.style.use('dark_background')
    plt.figure(figsize=(12, 6))
    plt.plot(equity_curve, color='cyan', label='Curva de Capital Walk-Forward')
    plt.title('Curva de Capital Walk-Forward')
    plt.xlabel('Ventanas de Prueba')
    plt.ylabel('Valor del Portafolio ($)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('walk_forward_equity_curve.png')
    print("Gráfico de la curva de capital Walk-Forward guardado como 'walk_forward_equity_curve.png'")


# --- Funciones de Simulación y Reportes ---

# Add a new parameter is_walk_forward to run_simulation
def run_simulation(data, params, is_optimization=False, is_walk_forward=False):
    # Use initial portfolio from params if available, otherwise use global config
    initial_portfolio = params.get('initial_portfolio', config['initial_portfolio'])
    portfolio = initial_portfolio if not is_walk_forward else data['price'].iloc[0] * 0 + initial_portfolio # Use initial portfolio only for full backtest
    position = 0.0
    in_trade = False
    is_short = False

    # Correctly initialize portfolio_history for Walk-Forward
    portfolio_history = [portfolio]
    trades_log = []

    # Indicators should be calculated before calling run_simulation
    # data = calculate_indicators(data.copy(), params) # Remove redundant calculation

    # Entrenar el modelo de ML
    ml_model, scaler = None, None
    if config['use_ml_filter']:
        try:
            # Select available features from the data, excluding the last row for target
            available_features = [f for f in config['ml_features'] if f in data.columns]
            if available_features:
                 # Pass data up to the point of training for ML
                ml_model, scaler = train_ml_model(data.iloc[:-1].copy(), available_features)
            else:
                if not is_optimization and not is_walk_forward:
                    print("Advertencia: No hay características de ML disponibles para entrenar.")
                config['use_ml_filter'] = False # Disable if no features

        except Exception as e:
            if not is_optimization and not is_walk_forward:
                print(f"Error al entrenar el modelo de ML: {e}. Desactivando filtro ML.")
            config['use_ml_filter'] = False

    strategy_func_map = {
        "crossover": check_crossover_strategy,
        "bollinger": check_bollinger_strategy,
        "ichimoku": check_ichimoku_strategy
    }
    strategy_func = strategy_func_map.get(params['strategy_name'])

    # Find the starting index after dropping NaNs during indicator calculation
    start_index = data.first_valid_index() if data.first_valid_index() is not None else 0
    if start_index > 0 and not is_optimization and not is_walk_forward:
        print(f"Iniciando simulación desde el índice {start_index} debido a NaNs en indicadores.")


    for i in range(start_index + 1, len(data)):
        current_price = float(data['price'].iloc[i])

        # Lógica de cierre para posiciones
        if in_trade:
            # Trailing Stop Loss
            if params.get('trailing_stop_percent'):
                if not is_short:
                    peak_price_in_trade = max(peak_price_in_trade, current_price)
                    trailing_stop_price = peak_price_in_trade * (1 - params['trailing_stop_percent'] / 100)
                    if current_price <= trailing_stop_price:
                        exit_price = current_price
                        pnl = (exit_price - entry_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                        portfolio += (position * exit_price)
                        trades_log.append({'type': 'Long - Trailing Stop Loss', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': True, 'date': data.iloc[i]['Date']})
                        in_trade = False
                else: # is_short
                    trough_price_in_trade = min(trough_price_in_trade, current_price)
                    trailing_stop_price = trough_price_in_trade * (1 + params['trailing_stop_percent'] / 100)
                    if current_price >= trailing_stop_price:
                        exit_price = current_price
                        pnl = (entry_price - exit_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                        portfolio += (position * (entry_price + pnl/position))
                        trades_log.append({'type': 'Short - Trailing Stop Loss', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': False, 'date': data.iloc[i]['Date']})
                        in_trade = False

            # Stop Loss y Take Profit
            if not in_trade: # Check again if trade was closed by trailing stop
                portfolio_history.append(portfolio + (position * current_price if not is_short else position * (entry_price - current_price)))
                continue

            if not is_short:
                if current_price <= stop_loss_price:
                    exit_price = current_price
                    pnl = (exit_price - entry_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                    portfolio += (position * exit_price)
                    trades_log.append({'type': 'Long - Stop Loss', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': True, 'date': data.iloc[i]['Date']})
                    in_trade = False
                elif current_price >= take_profit_price:
                    exit_price = current_price
                    pnl = (exit_price - entry_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                    portfolio += (position * exit_price)
                    trades_log.append({'type': 'Long - Take Profit', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': True, 'date': data.iloc[i]['Date']})
                    in_trade = False
            else: # is_short
                if current_price >= stop_loss_price:
                    exit_price = current_price
                    pnl = (entry_price - exit_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                    portfolio += (position * (entry_price + pnl/position))
                    trades_log.append({'type': 'Short - Stop Loss', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': False, 'date': data.iloc[i]['Date']})
                    in_trade = False
                elif current_price <= take_profit_price:
                    exit_price = current_price
                    pnl = (entry_price - exit_price) * position * (1 - config['commission_pct'] / 100) # Apply commission
                    portfolio += (position * (entry_price + pnl/position))
                    trades_log.append({'type': 'Short - Take Profit', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': False, 'date': data.iloc[i]['Date']})
                    in_trade = False

            if not in_trade: # Check again if trade was closed
                portfolio_history.append(portfolio)
                continue

        buy_signal, sell_signal = strategy_func(data, i, params)

        ml_prediction = 1 # Default to bullish if ML is off or fails
        if config['use_ml_filter'] and ml_model and scaler:
            try:
                features_to_predict = data.iloc[i][available_features].values.reshape(1, -1) # Use available_features
                # Check for NaNs or infs in features_to_predict before scaling
                if not np.isnan(features_to_predict).any() and not np.isinf(features_to_predict).any():
                     ml_prediction = ml_model.predict(scaler.transform(features_to_predict))[0]
                else:
                    ml_prediction = 1 # Default to bullish if features are invalid
                    if not is_optimization and not is_walk_forward:
                         print(f"Advertencia: Características de ML inválidas en la fecha {data.iloc[i]['Date'].date()}. Saltando predicción ML.")

            except Exception as e:
                if not is_optimization and not is_walk_forward:
                    print(f"Error durante la predicción de ML en la fecha {data.iloc[i]['Date'].date()}: {e}. Usando predicción por defecto.")
                ml_prediction = 1 # Default to bullish on error


        if buy_signal and not in_trade:
            if config['use_ml_filter'] and ml_prediction == 0:
                if not is_optimization and not is_walk_forward:
                    print(f"🚫 Señal de Compra de {params['strategy_name']} ignorada por filtro ML (predicción bajista) en {data.iloc[i]['Date'].date()}")
                pass # Skip trade if ML filter says no
            else:
                entry_price = float(data['price'].iloc[i])
                stop_loss_price = entry_price - (1.5 * float(data['atr'].iloc[i]))
                take_profit_price = entry_price + (params['take_profit_multiple'] * abs(entry_price - stop_loss_price))

                pnl_series_temp = pd.Series([trade['pnl'] for trade in trades_log if trade['is_long']]) # Only consider long trades for Kelly for longs
                win_rate = (len(pnl_series_temp[pnl_series_temp > 0]) / len(pnl_series_temp)) if len(pnl_series_temp) >= 10 else 0 # Require at least 10 trades for Kelly
                avg_win_loss = (pnl_series_temp[pnl_series_temp > 0].mean() / abs(pnl_series_temp[pnl_series_temp < 0].mean())) if len(pnl_series_temp[pnl_series_temp < 0]) >= 1 and len(pnl_series_temp[pnl_series_temp > 0]) >=1 else 0 # Require at least 1 winning and 1 losing trade for Kelly

                units = calculate_position_size(
                    portfolio_value=portfolio,
                    entry_price=entry_price,
                    stop_loss_price=stop_loss_price,
                    win_rate=win_rate,
                    avg_win_loss=avg_win_loss
                )

                if units > 1e-6 and units * entry_price <= portfolio * 0.99: # Ensure enough capital and minimum units
                    cost = units * entry_price
                    portfolio -= cost
                    position = units
                    in_trade = True
                    is_short = False
                    peak_price_in_trade = entry_price
                    if not is_optimization and not is_walk_forward:
                         print(f"🚀 Compra Larga ejecutada en {data.iloc[i]['Date'].date()} a ${current_price:.2f} (Tamaño: {units:.4f} unidades)")

                elif units > 1e-6: # If units > 0 but not enough capital
                     if not is_optimization and not is_walk_forward:
                        print(f"🚫 Fondos insuficientes para la compra larga de {units:.4f} unidades en {data.iloc[i]['Date'].date()}")


        if sell_signal and not in_trade:
            if config['use_ml_filter'] and ml_prediction == 1:
                 if not is_optimization and not is_walk_forward:
                    print(f"🚫 Señal de Venta de {params['strategy_name']} ignorada por filtro ML (predicción alcista) en {data.iloc[i]['Date'].date()}")
                 pass # Skip trade if ML filter says no
            else:
                entry_price = float(data['price'].iloc[i])
                stop_loss_price = entry_price + (1.5 * float(data['atr'].iloc[i]))
                take_profit_price = entry_price - (params['take_profit_multiple'] * abs(entry_price - stop_loss_price))

                pnl_series_temp = pd.Series([trade['pnl'] for trade in trades_log if not trade['is_long']]) # Only consider short trades for Kelly for shorts
                win_rate = (len(pnl_series_temp[pnl_series_temp > 0]) / len(pnl_series_temp)) if len(pnl_series_temp) >= 10 else 0 # Require at least 10 trades for Kelly
                avg_win_loss = (pnl_series_temp[pnl_series_temp > 0].mean() / abs(pnl_series_temp[pnl_series_temp < 0].mean())) if len(pnl_series_temp[pnl_series_temp < 0]) >= 1 and len(pnl_series_temp[pnl_series_temp > 0]) >=1 else 0 # Require at least 1 winning and 1 losing trade for Kelly


                units = calculate_position_size(
                    portfolio_value=portfolio,
                    entry_price=entry_price,
                    stop_loss_price=stop_loss_price,
                    win_rate=win_rate,
                    avg_win_loss=avg_win_loss
                )

                if units > 1e-6 and units * entry_price <= portfolio * 0.99: # Ensure enough capital and minimum units
                    position = units
                    in_trade = True
                    is_short = True
                    trough_price_in_trade = entry_price
                    if not is_optimization and not is_walk_forward:
                        print(f"🔻 Venta Corta ejecutada en {data.iloc[i]['Date'].date()} a ${current_price:.2f} (Tamaño: {units:.4f} unidades)")

                elif units > 1e-6: # If units > 0 but not enough capital
                     if not is_optimization and not is_walk_forward:
                        print(f"🚫 Fondos insuficientes para la venta corta de {units:.4f} unidades en {data.iloc[i]['Date'].date()}")


        # Update portfolio history at the end of each day
        if in_trade:
            if is_short:
                current_pnl = (entry_price - current_price) * position
                portfolio_history.append(portfolio + current_pnl)
            else:
                current_pnl = (current_price - entry_price) * position
                portfolio_history.append(portfolio + (position * entry_price) + current_pnl)
        else:
             portfolio_history.append(portfolio)


    # Cálculos finales
    # Ensure any open trade is closed at the last price for final portfolio calculation
    if in_trade:
        exit_price = float(data['price'].iloc[-1])
        if not is_short:
            pnl = (exit_price - entry_price) * position * (1 - config['commission_pct'] / 100)
            portfolio += (position * exit_price)
            trades_log.append({'type': 'Long - End of Simulation', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': True, 'date': data.iloc[-1]['Date']})
        else:
             pnl = (entry_price - exit_price) * position * (1 - config['commission_pct'] / 100)
             portfolio += (position * (entry_price + pnl/position))
             trades_log.append({'type': 'Short - End of Simulation', 'pnl': pnl, 'entry_price': entry_price, 'exit_price': exit_price, 'is_long': False, 'date': data.iloc[-1]['Date']})

    final_portfolio = portfolio # Portfolio already includes value of closed trades

    pnl_series = pd.Series([trade['pnl'] for trade in trades_log])

    total_trades = len(trades_log)

    # Ensure total_return is always calculated as a float
    initial_port_val = float(config['initial_portfolio']) if config['initial_portfolio'] is not None else 0.0
    final_port_val = float(final_portfolio) if final_portfolio is not None else initial_port_val # Use initial if final is None

    total_return = ((final_port_val - initial_port_val) / initial_port_val) * 100 if initial_port_val > 0 else 0.0


    if total_trades == 0:
         # Return safe default values if no trades
        return {'sharpe_ratio': 0.0, 'total_return': total_return, 'calmar_ratio': 0.0, 'drawdown': 0.0, 'profit_factor': 0.0, 'final_portfolio': final_port_val, 'total_trades': 0, 'trades_log': [], 'equity_curve': pd.Series([initial_port_val])}


    winning_trades_pnl = pnl_series[pnl_series > 0]
    losing_trades_pnl = pnl_series[pnl_series < 0]

    total_gain = winning_trades_pnl.sum()
    total_loss = abs(losing_trades_pnl.sum())
    profit_factor = total_gain / total_loss if total_loss > 1e-6 else float('inf') # Avoid division by near-zero

    total_pnl = pnl_series.sum()
    tax_paid = total_pnl * (config['tax_pct'] / 100) if total_pnl > 0 else 0
    final_portfolio_after_tax = final_port_val - tax_paid # Apply tax to the final portfolio value


    # Equity curve from history
    equity_curve = pd.Series(portfolio_history, index=data['Date'].iloc[start_index:].values if start_index < len(data) and 'Date' in data.columns else None) # Use dates for index, handle empty data and missing Date
    if equity_curve.empty: # Ensure at least one value if history is empty
        equity_curve = pd.Series([initial_port_val])

    peak = equity_curve.expanding(min_periods=1).max()
    # Handle potential division by zero if peak is zero or negative
    drawdown = ((equity_curve - peak) / peak)
    max_drawdown = drawdown.min()

    daily_returns = equity_curve.pct_change().dropna()
    sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252) if daily_returns.std() > 1e-6 else 0.0 # Avoid division by near-zero
    annualized_return = (1 + daily_returns.mean()) ** 252 - 1
    calmar_ratio = annualized_return / abs(max_drawdown) if abs(max_drawdown) > 1e-6 else float('inf') # Avoid division by near-zero


    return {
        'sharpe_ratio': sharpe_ratio,
        'total_return': total_return,
        'calmar_ratio': calmar_ratio,
        'drawdown': max_drawdown,
        'profit_factor': profit_factor,
        'final_portfolio': final_portfolio_after_tax,
        'total_trades': total_trades,
        'trades_log': trades_log,
        'equity_curve': equity_curve # Return equity curve for plotting
    }

def generate_report(results, symbol):
    """
    Genera el reporte final y los gráficos.
    """
    print("-" * 50)
    print("Reporte de Backtesting")
    print("-" * 50)
    print(f"Portafolio inicial: ${config['initial_portfolio']:.2f}")
    print(f"Portafolio final:   ${results['final_portfolio']:.2f}")
    print(f"Retorno total:      {results['total_return']:.2f}%")
    print("\n--- Métricas de Performance ---")
    print(f"Operaciones totales: {results['total_trades']}")
    print(f"Factor de Ganancia: {results['profit_factor']:.2f}")
    print(f"Drawdown Máximo:    {results['drawdown']:.2f}%")
    print(f"Ratio de Calmar:    {results['calmar_ratio']:.2f}")
    print(f"Ratio de Sharpe:    {results['sharpe_ratio']:.2f}")

    if not results['trades_log']:
        print("\nNo se realizaron operaciones. No se pueden generar gráficos de trades.")
        # Still generate equity curve if history is available
        if 'equity_curve' in results and not results['equity_curve'].empty:
             plt.style.use('dark_background')
             plt.figure(figsize=(12, 6))
             plt.plot(results['equity_curve'], color='lime', label='Curva de Capital')
             plt.title('Curva de Capital (Equity Curve)')
             plt.xlabel('Fecha')
             plt.ylabel('Valor del Portafolio ($)')
             plt.legend()
             plt.grid(True, linestyle='--', alpha=0.6)
             plt.tight_layout()
             plt.savefig('equity_curve.png')
             print("Gráfico de la curva de capital guardado como 'equity_curve.png'")
        return


    # Gráficos de Visualización
    # Use the equity curve returned from run_simulation
    equity_curve = results['equity_curve']

    # Gráfico 1: Curva de Capital (simulación completa)
    plt.style.use('dark_background')
    plt.figure(figsize=(12, 6))
    plt.plot(equity_curve, color='lime', label='Curva de Capital')
    plt.title('Curva de Capital (Equity Curve)')
    plt.xlabel('Fecha')
    plt.ylabel('Valor del Portafolio ($)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('equity_curve.png')
    print("Gráfico de la curva de capital guardado como 'equity_curve.png'")

    # Gráfico 2: Señales de entrada y salida
    # Need to re-download data for plotting the price chart with signals
    try:
        price_data = yf.download(symbol, period=config['time_period'], interval=config['time_interval'], auto_adjust=True)
        if 'Adj Close' in price_data.columns:
            price_data['price'] = price_data['Adj Close']
        else:
            price_data['price'] = price_data['Close']
        price_data.dropna(inplace=True)
        price_data.reset_index(drop=False, inplace=True)
    except Exception as e:
        print(f"Error al descargar datos para el gráfico de señales: {e}")
        price_data = pd.DataFrame() # Empty dataframe if download fails


    plt.figure(figsize=(12, 6))
    if not price_data.empty:
        plt.plot(price_data['Date'], price_data['price'], label='Precio de Cierre', color='royalblue')

        buy_trades = [t for t in results['trades_log'] if 'is_long' in t and t['is_long']]
        sell_trades = [t for t in results['trades_log'] if 'is_long' in t and not t['is_long']]

        # Mapeo de precios de entrada/salida a fechas en el dataframe original
        buy_dates = [t['date'] for t in buy_trades if 'date' in t]
        buy_prices = [t['entry_price'] for t in buy_trades if 'entry_price' in t]

        sell_dates = [t['date'] for t in sell_trades if 'date' in t]
        sell_prices = [t['entry_price'] for t in sell_trades if 'entry_price' in t] # Use entry_price for short sell signals

        # Also add exit points for visualization
        long_exit_trades = [t for t in results['trades_log'] if 'is_long' in t and t['is_long'] and 'exit_price' in t]
        long_exit_dates = [t['date'] for t in long_exit_trades if 'date' in t]
        long_exit_prices = [t['exit_price'] for t in long_exit_trades if 'exit_price' in t]

        short_cover_trades = [t for t in results['trades_log'] if 'is_long' in t and not t['is_long'] and 'exit_price' in t]
        short_cover_dates = [t['date'] for t in short_cover_trades if 'date' in t]
        short_cover_prices = [t['exit_price'] for t in short_cover_trades if 'exit_price' in t]


        if buy_trades:
            plt.scatter(buy_dates, buy_prices, marker='^', color='green', s=100, zorder=5, label='Entrada (Long)')
        if long_exit_dates:
             plt.scatter(long_exit_dates, long_exit_prices, marker='v', color='red', s=100, zorder=5, label='Salida (Long)')

        if sell_trades:
            plt.scatter(sell_dates, sell_prices, marker='v', color='red', s=100, zorder=5, label='Entrada (Short)')
        if short_cover_dates:
             plt.scatter(short_cover_dates, short_cover_prices, marker='^', color='green', s=100, zorder=5, label='Cobertura (Short)')


        plt.title(f'Señales de Trading para {symbol}')
        plt.xlabel('Fecha')
        plt.ylabel('Precio ($)')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.savefig('trade_signals.png')
        print("Gráfico de señales de trading guardado como 'trade_signals.png'")
        plt.show() # Ensure plot is displayed
    else:
        print("No se pudo generar el gráfico de señales de trading debido a un error al descargar datos.")


    # Gráfico 3: Visualización de Ichimoku (si se seleccionó)
    if config['strategy_name'] == "ichimoku":
        # Need to re-download data and calculate Ichimoku indicators for plotting
        try:
            ichimoku_data = yf.download(symbol, period=config['time_period'], interval=config['time_interval'], auto_adjust=True)
            if 'Adj Close' in ichimoku_data.columns:
                ichimoku_data['price'] = ichimoku_data['Adj Close']
            else:
                ichimoku_data['price'] = ichimoku_data['Close']
            ichimoku_data.dropna(inplace=True)
            ichimoku_data.reset_index(drop=False, inplace=True)

            # Recalculate Ichimoku indicators specifically for plotting
            high, low, close = ichimoku_data['High'], ichimoku_data['Low'], ichimoku_data['price']
            ichimoku_ind = ta.trend.IchimokuIndicator(high=high, low=low, window1=config['ichimoku_tenkan'], window2=config['ichimoku_kijun'], window3=config['ichimoku_senkou'])
            ichimoku_data['tenkan_sen'] = ichimoku_ind.ichimoku_conversion_line()
            ichimoku_data['kijun_sen'] = ichimoku_ind.ichimoku_base_line()
            ichimoku_data['senkou_span_a'] = ichimoku_ind.ichimoku_a()
            ichimoku_data['senkou_span_b'] = ichimoku_ind.ichimoku_b()
            ichimoku_data.dropna(subset=['tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b'], inplace=True)

        except Exception as e:
            print(f"Error al descargar o calcular indicadores Ichimoku para el gráfico: {e}")
            ichimoku_data = pd.DataFrame() # Empty dataframe if fails


        if not ichimoku_data.empty:
            plt.figure(figsize=(12, 6))
            plt.plot(ichimoku_data['Date'], ichimoku_data['price'], label='Precio de Cierre', color='white', alpha=0.8)
            plt.plot(ichimoku_data['Date'], ichimoku_data['tenkan_sen'], label='Tenkan Sen', color='red', linestyle='--')
            plt.plot(ichimoku_data['Date'], ichimoku_data['kijun_sen'], label='Kijun Sen', color='blue', linestyle='--')
            plt.fill_between(ichimoku_data['Date'], ichimoku_data['senkou_span_a'], ichimoku_data['senkou_span_b'], where=ichimoku_data['senkou_span_a'] > ichimoku_data['senkou_span_b'], color='green', alpha=0.3, label='Nube (Alcista)')
            plt.fill_between(ichimoku_data['Date'], ichimoku_data['senkou_span_a'], ichimoku_data['senkou_span_b'], where=ichimoku_data['senkou_span_a'] < ichimoku_data['senkou_span_b'], color='red', alpha=0.3, label='Nube (Bajista)')

            # Use the trade dates and prices from the simulation results
            buy_trades = [t for t in results['trades_log'] if 'is_long' in t and t['is_long']]
            sell_trades = [t for t in results['trades_log'] if 'is_long' in t and not t['is_long']]

            buy_dates = [t['date'] for t in buy_trades if 'date' in t]
            buy_prices = [t['entry_price'] for t in buy_trades if 'entry_price' in t]

            sell_dates = [t['date'] for t in sell_trades if 'date' in t]
            sell_prices = [t['entry_price'] for t in sell_trades if 'entry_price' in t]


            plt.scatter(buy_dates, buy_prices, marker='^', color='lime', s=100, zorder=5, label='Señal de Compra')
            plt.scatter(sell_dates, sell_prices, marker='v', color='crimson', s=100, zorder=5, label='Señal de Venta')


            plt.title(f'Estrategia de Ichimoku Cloud para {symbol}')
            plt.xlabel('Fecha')
            plt.ylabel('Precio ($)')
            plt.legend()
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.tight_layout()
            plt.savefig('ichimoku_signals.png')
            print("Gráfico de señales de Ichimoku guardado como 'ichimoku_signals.png'")
            plt.show() # Ensure plot is displayed
        else:
            print("No se pudo generar el gráfico de Ichimoku debido a un error.")


# --- Lógica Principal del Programa ---
if __name__ == "__main__":
    symbol = setup_menu()

    # Ensure initial_portfolio is a float before any analysis function is called
    try:
        config['initial_portfolio'] = float(config.get('initial_portfolio', 10000))
    except (ValueError, TypeError):
        print(f"Advertencia: El capital inicial '{config.get('initial_portfolio')}' no es un número válido. Usando el valor por defecto de 10000.")
        config['initial_portfolio'] = 10000.0


    try:
        data = yf.download(config['symbol'], period=config['time_period'], interval=config['time_interval'], auto_adjust=True)
        if 'Adj Close' in data.columns:
            data['price'] = data['Adj Close']
        else:
            data['price'] = data['Close']
        data.dropna(inplace=True)
        data.reset_index(drop=False, inplace=True)

    except Exception as e:
        print(f"Error al descargar datos para {config['symbol']}: {e}")
        exit()

    # Calculate indicators once after data download
    data = calculate_indicators(data.copy(), config)
    data.dropna(inplace=True) # Drop NaNs again after calculating all indicators

    if config['advanced_analysis'] == "genetic_algorithm":
        best_params = run_optimization_ga(data, config)
        final_params = {**config, **best_params}
        results = run_simulation(data, final_params) # Run final simulation with best params
        generate_report(results, symbol)

    elif config['advanced_analysis'] == "sensitivity":
        run_sensitivity_analysis(data, config)

    elif config['advanced_analysis'] == "walk_forward":
        run_walk_forward_analysis(data, config)

    else: # Sin análisis avanzado
        results = run_simulation(data, config)
        generate_report(results, symbol)

--- Configuración del Backtesting Definitivo ---
Por favor, ingrese los siguientes parámetros. Presione ENTER para usar los valores por defecto.
--------------------------------------------------
Símbolo del Activo (ej: BTC-USD, AAPL)
[default: BTC-USD]: BTC-USD
Capital Inicial [default: 10000]: 1000
Seleccione una estrategia:
1. Crossover de EMA (para mercados en tendencia)
2. Bollinger Bands (para reversión a la media)
3. Ichimoku Cloud (análisis completo de tendencia)
(1/2/3) [default: 3]: 3
Seleccione el método de gestión de capital:
1. Riesgo Fijo por Operación (simple)
2. Criterio de Kelly Fraccional (avanzado, basado en el historial)
(1/2) [default: 2]: 2
Fracción de Kelly (0.1 a 1.0) [default: 0.5]: 0.5

--- Análisis Avanzado (Seleccione solo uno) ---
Seleccione una opción de análisis avanzado:
1. Algoritmo Genético (optimiza parámetros automáticamente)
2. Análisis de Sensibilidad (mapa de calor)
3. Análisis Walk-Forward (validación rigurosa)
4. Sin Análisis Avanzado
(1/2/3/4) 

[*********************100%***********************]  1 of 1 completed



Iniciando Análisis Walk-Forward...
Debug: Initial Walk-Forward Equity: 1000.0 (Type: <class 'float'>)

--- Walk-Forward Window: N/A to N/A ---

Iniciando optimización con Algoritmo Genético...

✅ Optimización terminada. Mejor resultado (Ratio de Sharpe: 0.19) con los parámetros:
{'ichimoku_tenkan': 7, 'ichimoku_kijun': 27}


IndexError: single positional indexer is out-of-bounds